<a href="https://colab.research.google.com/github/jneey2000/ML/blob/main/2_The_engine_of_neural_networks_gradient_based_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

'2.15.0'

# The engine of neural networks: gradient-based optimization

Each neural layer from our first network example **transforms its input data as follows**  
(첫 번째 신경망 예제에 있는 **<font color="orange">각 층(dense layer)</font>은 <font color="orange">입력 데이터</font>를 다음과 같이 변환함**):

<font color="orange">**output = relu(dot(W, input) + b)**</font>

### Weights (Trainable parameters)
- In this expression, **W and b are tensors** that are **attributes** of the layer.  
(이 식에서 텐서 W와 b는 층의 속성처럼 볼 수 있음)
- They’re called **the weights** or **trainable parameters** of the layer (the kernel and bias attributes, respectively).  
(<font color="orange">**가중치(weight)**</font> 또는 <font color="orange">**훈련되는 파라미터(trainable parameter)**</font>라고 부름.  
 * W: kernel(커널)
 * b: bias (편향)
- These weights contain the **information learned by the network** from exposure to training data.  
(<U>**이런 가중치에는**</U> 훈련 데이터를 신경망에 노출시켜서 <U><font color="orange">**학습된 정보**</font></U>가 담겨 있음)

### Training and Random initialization

- Initially, these weight matrices are filled with **small random values** (a step called **random initialization**). [Keras-Initializers](https://keras.io/initializers/).  
(초기에는 가중치 행렬이 작은 난수로 채워져 있음: <font color="orange">**무작위 초기화 (random initialization) 단계**</font>라고 부름)
- Of course, there’s no reason to expect that relu(dot(W, input) + b), when W and b are random, will yield any useful representations.  
(**초기 W와 b가 random values일 때** relu(dot(W, input) + b)가 **<U>유용한 어떤 표현을 만들 것이라고 기대할 수 없음</U>**.)
- The resulting representations are meaningless—but they’re a starting point.  
(**즉, 의미 없는 표현이 만들어짐. 하지만 시작점임**.)

- What comes next is to gradually adjust these weights, based on a feedback signal.  
(그다음에는 <font color="orange">**피드백 신호에 기초하여 가중치가 점진적으로 조정**</font>될 것임)
- This gradual adjustment, also called **training**, is basically the learning that machine learning is all about.  
(이런 점진적인 조정 또는 훈련(Training)이 머신러닝 학습의 핵심임)

(Definition) <font color="red">**Training (Learning, 학습)이란?**:  
<U>**Training loss가 최소가 되도록 파라미터(가중치)를 점진적으로 조정하는 것**</U></font>

This happens within what’s called a **training loop**, which works as follows.  
(훈련은 다음과 같은 훈련 반복 루프(training loop)안에서 일어남)  
Repeat these steps in a loop, as long as necessary (필요한 만큼 반복 루프 안에서 이런 단계가 반복됨):

1. Draw a batch (or a minibatch)) of training samples x and corresponding targets y.  
(<font color="orange">**훈련 샘플 x와 이에 상응하는 타깃 y의 배치를 추출함**</font>)
1. Run the network on x (a step called the **forward pass**) to obtain predictions y_pred.  
(<font color="orange">**x를 사용하여 네트워크를 실행하고(정방향 패스(forward pass) 단계), <U>예측 y_pred를 구함</U>**</font>)
1. **Compute the loss** of the network on the batch, a measure of the mismatch between y_pred and y.  
(<font color="orange">**y_pred와 y의 차이를 측정하여 이 배치에 대한 네트워크의 <U>손실을 계산함</U>**</font>)
1. **Update all weights** of the network in a way that slightly reduces the loss on this batch.  
(<font color="orange">**배치에 대한 <U>손실이 조금 감소</U>되도록 네트워크의 <U>모든 가중치를 업데이트</U>함(Backward pass 단계)**</font>)

<div>
<img src="https://drive.google.com/uc?export=view&id=1PvWGgY_PAHt2viEV127N2WrLXUbNNYrq" width="600"/>
</div>


<div>
<img src="https://drive.google.com/uc?export=view&id=1Dh8k87WdElkYP0Td6b56KLit2auMzxMv" width="500"/>
</div>


- You’ll eventually end up with a network that has a very low loss on its training data: a low mismatch between predictions y_pred and expected targets y.  
(<font color="orange">**결국 훈련 데이터에서 네트워크의 손실, 즉 예측 y_pred와 target y의 <U>오차가 매우 작아질 것</U>임**</font>)
- The network has “learned” to map its inputs to correct targets.  
(<font color="orange">**이 네트워크는 <U>입력에 정확한 타깃을 매핑</U>하는 것을 학습함**</font>)
- From afar, it may look like magic, but when you reduce it to elementary steps, it turns out to be simple.

### Updating weigths of a neural network

- Step 1 sounds easy enough—just I/O code (Input/Output).  
(**1단계는 단순히 입출력 코드임**)
- Steps 2 and 3 are merely the application of a handful of tensor operations, so you could implement these steps purely from what you learned in the previous section.  
(**2단계(y_pred 예측)와 3단계(손실계산)는 몇 개의 텐서 연산으로 구현 가능**)
- The difficult part is step 4: updating the network’s weights.  
(**<U>가장 어려운 부분은 네크워크의 가중치를 업데이트하는 4단계임</U>**)
- Given an individual weight coefficient in the network, **how can you compute whether the coefficient should be increased or decreased, and by how much?**  
<font color="orange">(**Q: <U>개별적인 가중치 값</U>이 있을 때 값이 증가해야 할지 감소해야 하지, 또 얼마큼 업데이트해야 할지 어떻게 알 수 있을까요?**)</font>

### Simple example:
- One naive solution would be to freeze all weights in the network except the one scalar coefficient being considered, and try different values for this coefficient.  
(**한 가지 간단한 방법은 네트워크 가중치 행렬의 원소를 모두 고정하고 관심 있는 하나만 다른 값을 적용해 보는 것임**)
- Let’s say the initial value of the coefficient is 0.3.  
(**이 가중치의 초깃값이 0.3이라고 가정하자**)
- After the forward pass on a batch of data, the loss of the network on the batch is 0.5.   
(**배치 데이터를 정방향 패스를 통과시킨 후 네트워크의 손실이 0.5가 나왔음**)
- If you change the coefficient’s value to 0.35 and rerun the forward pass, the loss increases to 0.6.  
(**이 가중치 값을 0.35로 변경하고 다시 정방향 패스를 실행했더니 손실이 0.6으로 증가함**)
- But if you lower the coefficient to 0.25, the loss falls to 0.4.  
(**반대로 0.25로 줄이면 손실이 0.4로 감소함**)
- In this case, it seems that updating the coefficient by -0.05 would contribute to minimizing the loss.   
(**이 경우에 가중치를 -0.05만큼 업데이트한 것이 손실을 줄이는 데 기여한 것으로 보임**)
- This would have to be repeated for all coefficients in the network.  
(**이런 식으로 네트워크의 모든 가중치에 반복함**)

### Much better to use <font color="orange">**Gradient Descent Method(경사하강법)**</font>:

- But such an approach would be horribly inefficient, because you’d need to compute two forward passes (which are expensive) for every individual coefficient (of which there are many, **usually thousands and sometimes up to millions**).  
(**이런 접근 방식은 모든 가중치 행렬의 원소마다 두 번의 비용이 큰 정방향 패스를 계산해야 하므로 <U>엄청나게 비효율적</U>임.  
<U>보통 수천에서 경우에 따라 수백만 개의 많은 가중치가 있음</U>**)
- A much better approach is to take advantage of the fact that **all operations used in the network are differentiable**  
(**신경망에 사용된 <font color="orange">모든 연산이 미분 가능(differentiable)하다</font>는 장점을 사용하자**)
- **Compute the gradient of the loss** with regard to the network’s coefficients (weights).   
(<font color="orange">**신경망의 가중치에대한 <U>손실의 그래디언트(gradient)를 계산하는 것</U>이 훨씬 더 좋은 방법임**</font>)
- You can then move the coefficients in **the opposite direction from the gradient**, thus decreasing the loss.  
(<font color="orange">**<U>그래디언트의 반대 방향</U>으로 가중치를 이동하면 <U>손실이 감소</U>됨**</font>)

## What’s a derivative? (변화율)

- Consider a **continuous, smooth function** f(x) = y, mapping a real number x to a new real number y.  
(**실수 x를 새로운 실수 y로 매핑하는 연속적이고 매끄러운 함수 f(x)=y를 생각해 보자**)
- Because the function is continuous, a small change in x can only result in a small change in y—that’s the intuition behind continuity.  
(**이 함수가 연속적이므로 x를 조금 바꾸면 y가 조금 변경될 것임**)
- Let’s say you increase x by a small factor epsilon_x: this results in a small epsilon_y change to y:  
(**x를 작은 값 epsilon_x만큼 증가시켰을 때 y가 epsilon_y만큼 바뀐다고 말할 수 있음**)

**f(x + epsilon_x) = y + epsilon_y**

- In addition, because the function is smooth (its curve doesn’t have any abrupt angles), when epsilon_x is small enough, around a certain point p, it’s possible to approximate f as a linear function of slope a  
(**또 이 함수가 매끈하므로 epsilon_x가 충분히 작다면 어떤 포인트 p에서 a의 선형함수 f를 근사할 수 있음.**)
- epsilon_y becomes a * epsilon_x:  
 **f(x + epsilon_x) = y + a * epsilon_x**

- Obviously, this linear approximation is valid only when x is close enough to p.  
(이 선형적인 근사는 x가 p에 충분히 가까울 때 유효함)

<div>
<img src="https://drive.google.com/uc?export=view&id=1m3J_PLi0FO9lzgCTvJN2uzDgCf5KvBFW" width="500"/>
</div>

- The slope a is called the **derivative** of f in p.  
(**이 기울기를 p에서 f의 변화율(derivative)라고 함**)
- If a is negative, it means a small change of x around p will result in a decrease of f(x) (as shown in figure 2.10); and if a is positive, a small change in x will result in an increase of f(x).   
(**이는 기울기 a가 음수일 때 양수 x만큼 조금 이동하면 f(x)가 감소함.  
기울기 a가 양수일 때는 음수 x만큼 조금 이동하면 f(x)가 감소됨**)
- Further, the absolute value of a (the magnitude of the derivative) tells you how quickly this increase or decrease will happen.  
(**<font color="orange"><U>기울기 a의 절대값(변화율의 크기)</U>은 이런 증가나 감소가 얼마나 빠르게 일어날지 알려 줌</font>**)

<div>
<img src="https://drive.google.com/uc?export=view&id=10-C8QJOwqxlsUh3zt-nTPNHkUeE_2P5S" width="600"/>
</div>

- For every **differentiable function f(x)** (differentiable means “can be derived”: for example, smooth, continuous functions can be derived), **there exists a derivative function f'(x)** that maps values of x to the slope of the local linear approximation of f in those points.  
(**모든 미분 가능한 함수 f(x)에 대해 x의 값을 f의 국부적인 선형 근사인 그 지점의 기울기로 매핑하는 변화율 함수 f'(x)가 존재함**)
- **For instance, the derivative of cos(x) is -sin(x), the derivative of f(x) = a * x is f'(x) = a, and so on.**

- If you’re trying to update x by a factor epsilon_x in order to minimize f(x), and you know **the derivative of f**, then your job is done:
- the derivative completely describes **how f(x) evolves as you change x**.
- If you want to reduce the value of f(x), you just need to move x a little in the opposite direction from the derivative.  
(**<font color="orange">f(x)의 값을 감소 시키고 싶다면 x를 변화율의 방향과 반대로 조금 이동해야 함</font>**)

## Derivative of a tensor operation: the gradient

- A **gradient** is **the derivative of a tensor operation**.  
(<font color="orange">**그래디언트는 텐서 연산의 변화율임**</font>)  
- It’s **the generalization of the concept of derivatives** to **functions of multidimensional inputs**:  
that is, to functions that take tensors as inputs.  
(**즉, 다차원 입력, <font color="orange"><U>텐서를 입력으로 받는 함수에 변화율</U></font> 개념을 확장시킨 것임**)

- Consider an input vector x, a matrix W, a target y, and a loss function loss.
- You can use W to compute a target candidate y_pred  
(W를 사용하여 타깃의 예측 y_pred를 계산)
- Compute the loss, or mismatch, between the target candidate y_pred and the target y  
(손실 (타깃 예측 y_pred와 타깃 y 사이의 오차)를 계산 가능):



```
y_pred = dot(W, x)
loss_value = loss(y_pred, y)
```



If the data inputs x and y are frozen, then this can be interpreted as **a function mapping values of W** to loss values  
(입력 데이터 x와 y가 고정되어 있다면 **이 함수는 W를 손실 값에 매핑하는 함수**로 볼 수 있음):

<font color="orange">**loss_value = f(W)**</font>

- Let’s say the current value of W is W0.  
(**W의 현재 값을 W0라고 하자**)
- Then **the derivative of f in the point W0** is a **tensor gradient(f)(W0)** with the same shape as W, where each coefficient gradient(f) (W0)[i, j] indicates the direction and magnitude of the change in loss_value you observe when modifying W0[i, j].  
(**포인트 W0에서 f의 변화율은 W와 같은 크기의 텐서인 gradient(f)(W0)임.  
이 텐서의 각 원소 gradient(f) (W0)[i, j]는 W0[i, j]를 변경했을 때 loss_value가 바뀌는 방향과 크기를 나타냄**)
- That tensor gradient(f)(W0) is the gradient of the function **f(W) = loss_value** in W0.  
(<font color="orange"> **텐서 gradient(f)(W0)가 W0에서 함수 f(W) = loss_value의 그래디언트임**</font>)

- You saw earlier that the derivative of a function f(x) of a single coefficient can be interpreted as the **slope** of the curve of f.  
(**함수 f(x)의 변화율(derivative) 하나는 곡선 f의 기울기임**)
- Likewise, gradient(f)(W0) can be interpreted as the tensor describing the **curvature** of f(W) around W0.  
(비슷하게, **gradient(f)(W0)는 W0에서 f(W)의 기울기를 나타내는 텐서임**)

- For this reason, in much the same way that, for a function f(x), you can reduce the value of f(x) by moving x a little in the opposite direction from the derivative, with a function f(W) of a tensor, you can **reduce f(W) by moving W in the opposite direction from the gradient**:  
(<font color="orange">**그래디언트의 반대 방향으로 W를 움직이면 f(W)의 값을 줄일 수 있음**</font>)
- For example, <font color="orange">**W1 = W0 - step * gradient(f)(W0)**</font>
- Where **step** (or also called "**learning rate**")is a small scaling factor)  
(<font color="orange">**Step (learning rate, 학습률)은 파라미터의 업데이트의 양을 조절하는 값임 (예, 0.001)**</font>

- That means going against the curvature, which intuitively should put you lower on the curve.  
(**기울기가 작아지는 곡면의 낮은 위치로 이동된다는 의미임.**)
- Note that the scaling factor step is needed because gradient(f)(W0) only approximates the curvature when you’re close to W0, so you don’t want to get too far from W0.  
(<font color="orange">gradient(f)(W0)는 W0에 아주 가까이 있을 때 기울기를 근사한 것이므로  
 **W0에서 너무 크게 벗어나지 않기 위해 스케일링 비율 step (learning rate)이 필요함**<font>)


## Stochastic gradient descent  

**요약:  
미분가능한 함수가 있으면 이론적으로 함수의 최솟값은 미분이 0이되는 지점에서 찾음.  
<U>하지만 신경망에서는 파라미터가 수천개에서 수천만개이므로 해석적으로 찾기 어려움.  
따라서 SGD 방법으로 찾음</U>**

- Given a differentiable function, it’s theoretically possible to find its minimum analytically:  
(**<U>미분 가능한 함수</U>가 주어지면 이론적으로 이 <U>함수의 최솟값</U>을 해석적으로 구할 수 있음**)
- it’s known that a function’s minimum is **a point where the derivative is 0**  
(**함수의 최솟값은 변화율이 0인 지점임**)  
- So all you have to do is find all the points where the derivative goes to 0 and check for which of these points the function has the lowest value.  
(**따라서 변화율이 0이 되는 지점을 모두 찾고 이 중에서 어떤 포인트의 함수 값이 가장 작은지 확인하면 됨**)


- Applied to a neural network, that means finding analytically the combination of weight values that yields the smallest possible loss function.  
(**신경망에 적용하면 가장 작은 손실 함수의 값을 만드는 가중치의 조합을 해석적으로 찾는 것을 의미함**)
- This can be done by **solving the equation gradient(f)(W) = 0 for W**.  
(**이는 식 gradient(f)(W) = 0을 풀면됨**)
- This is a polynomial equation of N variables, where N is the number of coefficients in the network.  
(**N이 네트워크의 가중치 개수라고 하면 위 식은 N개의 변수로 이루어진 다항식임**)
- Although it would be possible to solve such an equation for N = 2 or N = 3, doing so is **intractable for real neural networks**, where the number of parameters is never less than a few thousand and can often be **several tens of millions**.  
(**N=2나 N=3인 식을 푸는 것은 가능 하지만  
<font color="orange">수천개에서 수천만개의 파라미터를 가진 신경망에서 해석적으로 푸는 것은 어려움</font>**)

### **Mini-batch stochastic gradient descent (mini-batch SGD)**
- Instead, you can use the four-step algorithm outlined at the beginning of this section:  
(**앞서 설명한 <U>알고리즘 네 단계</U>를 사용할 수 있음**)
- Modify the parameters little by little based on the current loss value on a **random batch of data**.  
(**<font color="orange">랜덤한 배치 데이터</font>에서 현재 손실 값을 토대로 하여 조금씩 파라미터를 수정하는 것임**)
- Because you’re dealing with a differentiable function, you can compute its gradient,  
which gives you an efficient way to implement step 4.  
(**<U>미분 가능한 함수</U>를 가지고 있으므로 <U>그래디언트를 계산</U>하여 step 4를 효율적으로 구현할 수 있음**)
- If you update the weights in the opposite direction from the gradient, the loss will be a little less every time  
(**<U><font color="orange">그래디언트의 반대방향</font></U>으로 가중치를 업데이트하면 <U><font color="orange">손실이 매번 조금씩 감소할 것</font></U>임**):

1. Draw a batch of training samples x and corresponding targets y.  
(**훈련 샘플 배치 x와 이에 상응하는 타깃 y를 추출함**)
1. Run the network on x to obtain predictions y_pred.
(**x로 네트워크를 실행하고 예측 y_pred를 구함**)
1. Compute the loss of the network on the batch, a measure of the mismatch between y_pred and y.  
(**이 배치에서 y_pred와 y사이의 오차를 측정하여 네트워크의 손실을 계산함**)
1. Compute the gradient of the loss with regard to the network’s parameters (a backward pass).  
(**네트워크의 파라미터에 대한 손실 함수의 그래디언트를 계산함(역방향 패스 (backward pass)**)
1. Move the parameters a little in the opposite direction from the gradient—for example **W -= step * gradient**—thus reducing the loss on the batch a bit.  
(**그래디언트의 반대 방향으로 파라미터를 조금 이동 시킴**.  
 **예를 들어 <font color="orange">W -= step * gradient</font>
처럼하면 배치에 대한 손실이 조금 감소할 것임**)

- What I just described is called "**mini-batch stochastic gradient descent (mini-batch SGD)**".  
(이것을 <font color="orange">**미니 배치 확률적 경사 하강법(mini-batch stochastic gradient descent (mini-batch SGD))**</font>라고 함)
- The term "**stochastic**" refers to the fact that each batch of data is drawn at random (stochastic is a scientific synonym of random).  
(**<U>확률적(stochastic)</U>이란 단어는 각 배치 데이터가 무작위로 선택된다는 의미임**)
- Figure 2.11 illustrates what happens in 1D, when the network has only one parameter and you have only one training sample.

<div>
<img src="https://drive.google.com/uc?export=view&id=125AJnZQ_n9C94kqp9D_Jmo-d5rC8nm5J" width="600"/>
</div>

### Learning rate

- As you can see, intuitively it’s **important to pick a reasonable value for the step factor (learning rate)**.  
(<font color="red">딥러닝 학습에서 **learning rate (step)을 적절히 고르는 것이 매우 중요함**</font>)
- If it’s **too small**, the descent down the curve will take many iterations, and it could get **stuck in a local minimum**.  
(<font color="orange">**LR이 너무 작으면 곡선을 따라 내려가는 데 <U>너무 많은 반복(iteration)</U>이 필요하고 <U>지역 최솟값(local minimum)</U>에 갇힐 수 있음**</font>)
- If step is **too large**, your updates may end up taking you to **completely random** locations on the curve.  
(<font color="orange">**LR이 너무 크면 손실 함수 곡선에서 완전히 임의의 위치로 이동시킬 수 있음**</font>)

### True SGD, mini-batch SGD, and batch SGD

-  Batch gradient descent works by calculating the gradient of the loss function with respect to the model's parameters for the entire training dataset.  
<font color="orange">**Batch GD (BGD): 전체 학습 데이터를 하나의 batch(묶음)으로 묶어 학습**</font>   
 **전체 데이터를 모두 한 번에 처리하여 메모리가 많이 필요하다는 단점이 있음**
- Note that a variant of the mini-batch SGD algorithm would be to draw **a single sample and target at each iteration**, rather than drawing a batch of data. This would be **true SGD** (as opposed to mini-batch SGD):  
<font color="orange">**True SGD: 매 interation마다 하나의 샘플로 학습**</font>  
 **학습이 안될 수 있는 단점이 있음**
- Alternatively, going to the opposite extreme, you could run every step on **all data** available, which is called **SGD**. Each update would then be more accurate, but far more expensive.  
<font color="orange">**SGD (GD): 매 interation마다 모든 샘플로 학습**</font>  
 **더 정확하게 업데이트되지만 더많은 비용이 들고 local minimum에 빠질 수 있음**
- The efficient compromise between these two extremes is to use mini-batches of reasonable size.  
<font color="orange">**mini-batch SGD: 매 interation마다 1개보단 많고 training samples의 수보다는 적은 수의 샘플로 학습**</font>  
 **적절한 미니배치 사이즈를 찾아야함. 하이퍼파라미터임.**


- 그림 2-11은 1D 파라미터 공간에서 경사 하강법을 설명하고 있지만 매우 고차원 공간에서 경사하강법을 사용하게 됨
- 그래서 초고차원 공간을 시각화하기가 어려움.

<div>
<img src="https://drive.google.com/uc?export=view&id=1P3oUPDY7_f2Ekb4ls7zgdWnCswtoKUXp" width="500"/>
</div>

### Opimization methods or Optimizers

- Additionally, there exist multiple variants of SGD that differ by **taking into account previous weight updates when computing the next weight update**, rather than just looking at the current value of the gradients.  
 **업데이트할 가중치를 계산할 때 현재 그래디언트 값만 보지 않고 이전에 업데이트된 가중치를 여러 가지 방식으로 고려하는 SGD 변종이 많이 있음.**
- There is, for instance, **SGD with momentum**, as well as **Adagrad, RMSProp**, and several others. Such variants are known as **optimization methods** or **optimizers**.  
예를 들어 **SGD with momentum**, **Adagrad, RMSProp** 등이 있음.

- In particular, the concept of **momentum**, which is used in many of these variants, deserves your attention.  
<font color="orange">**모멘텀(momentum)이라는 개념은 매우 중요**</font>
- Momentum addresses two issues with SGD: **convergence speed and local minima**.  
<font color="orange">**모멘텀(momentum)은 SGD에 있는 2가지 문제점인 <U>수렴 속도와 지역 최솟값</U>을 해결함**</font>
- Consider figure 2.13, which shows the curve of a loss as a function of a network parameter.

<div>
<img src="https://drive.google.com/uc?export=view&id=1kRH4FO6HqYPB01spJNrSogoVQT2pjlIj" width="600"/>
</div>

- As you can see, around a certain parameter value, there is a **local minimum**: around that point, moving left would result in the loss increasing, but so would moving right.  
(**어떤 파라미터 값에서 local minimum에 도달함.  
그 지점 근체에서는 왼쪽으로 이동해도 손실이 증가하고,  
오른쪽으로 이동해도 손실이 증가함**)
- If the parameter under consideration were being optimized via **SGD with a small learning rate**, then the optimization process would get stuck at the local minimum instead of making its way to the global minimum.  
(**대상 파라미터가 작은 학습률을 가진 SGD로 최적화되었다면  
최적화 과정이 global minimum으로 향하지 못하고  
 <U>local minimum에 갇히게 될 것임</U>**)

### Momentum

- You can avoid such issues by using momentum, which draws inspiration from physics  
(<font color="orange">**물체가 한 방향으로 지속적으로 변동하려는 경향**</font>).  
A useful mental image here is to think of the optimization process as a small ball rolling down the loss curve.  
(**여기에서 최적화 과정을 손실 곡선 위로 작은 공을 굴리는 것을 생각하면 쉽게 이해할 수 있음**)
- If it has enough momentum, the ball won’t get stuck in a ravine and will end up at the global minimum.  
(**모멘텀이 충분하면 공이 골짜기에 갇히지 않고 전역 최솟값에 도달할 것임**)
- **Momentum** is implemented by moving the ball at each step based not only on the current slope value (current acceleration) but also on the current velocity (resulting from past acceleration).  
(**모멘텀은 현재 기울기 값(현재 가속도)뿐만 아니라 (과거의 가속도로 인한) 현재 속도를 함께 고려하여 각 단계에서 공을 움직임**)
- In practice, this means **updating the parameter w based not only on the current gradient value but also on the previous parameter update**, such as in this naive implementation  
(실전에 적용할 때는 <font color="orange">**현재 그래디언트 값뿐만 아니라 이전에 업데이트한 파라미터에 기초하여 파라미터 W를 업데이트함. 즉, 이전의 이동 방향(속도)을 함께 고려해서 마치 공이 굴러가던 관성을 적용해주는 것처럼 작동할 수 있도록 해줌**</font>:



```
past_velocity = 0.
momentum = 0.1
while loss > 0.01:
    w, loss, gradient = get_current_parameters()
    velocity = past_velocity * momentum - learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
```



## Chaining derivatives: the Backpropagation algorithm

- In the previous algorithm, we casually assumed that because a function is differentiable, we can explicitly compute its derivative.
- In practice, a neural network function consists of many tensor operations chained together, each of which has a simple, known derivative.
- For instance, this is a network f composed of three tensor operations, a, b, and c, with weight matrices W1, W2, and W3:

f(W1, W2, W3) = a(W1, b(W2, c(W3)))

- Calculus tells us that such a chain of functions can be derived using the following identity, called the **chain rule: f(g(x)) = f'(g(x)) * g'(x)**.
- Applying the chain rule to the computation of the gradient values of a neural network gives rise to an algorithm called **Backpropagation** (also sometimes called reverse-mode differentiation).
- Backpropagation starts with the final loss value and works backward from the top layers to the bottom layers, applying the chain rule to compute the contribution that each parameter had in the loss value.

- Nowadays, and for years to come, people will implement networks in modern frameworks that are capable of **symbolic differentiation**, such as TensorFlow.
- This means that, given a chain of operations with a known derivative, they can compute a gradient function for the chain (by applying the chain rule) that maps network parameter values to gradient values.
- When you have access to such a function, the backward pass is reduced to a call to this gradient function.
- Thanks to symbolic differentiation, you’ll never have to implement the Backpropagation algorithm by hand.
- For this reason, we won’t waste your time and your focus on deriving the exact formulation of the Backpropagation algorithm in these pages.
- All you need is a good understanding of how gradient-based optimization works.

# Looking back at our first example



## Input data:

In [ ]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

- The input images are stored in **Numpy tensors**, which are here formatted as float32 tensors of shape (60000, 784) (training data) and (10000, 784) (test data), respectively.  
(입력 이미지는 Numpy 텐서에 저장되며, 여기서는 각각 (60000, 784)(훈련 데이터) 및 (10000, 784)(테스트 데이터) 모양의 float32 텐서로 형식이 지정됨)

참고) Tensor는 데이터를 담는 다차원 배열(다차원 행렬)을 의미함.
- 0차원 / 스칼라(scalar) / 7, 3.14   
- 1차원 / 벡터(vector) / [1, 2, 3]
- 2차원 / 행렬(matrix) / [[1, 2]. [3, 4]]
- 3차원 / 텐서(tensor) / [[[...]]]  e.g., 컬러이미지
- 4차원 / 고차원텐서 / [batch, height, width, channel]  e.g., 딥러닝 모델 입력 형태

## Architecture of Our Neural Network:

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

- This network consists of a chain of two Dense layers, that each layer applies a few simple tensor operations to the input data, and that these operations involve weight tensors.   
(이 네트워크는 두 개의 Dense 레이어 체인으로 구성됨. 각 레이어는 입력 데이터에 몇 가지 간단한 텐서 연산을 적용하고 이러한 작업에는 가중치 텐서가 포함됨)
- Weight tensors, which are attributes of the layers, are where the knowledge of the network persists.  
(레이어의 속성인 가중치 텐서는 네트워크에 대한 지식이 유지되는 곳임)


## Network-compilation step:

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy', # regression 문제라면 'mean_squared_error' 또는 'mse' 등을 쓸 수 있음
                metrics=['accuracy']) # regression 문제라면 'mae', 'mse' 등을 쓸 수 있음

- Loss function : categorical_crossentropy (the loss function that’s used as a feedback signal for learning the weight tensors, and which the training phase will attempt to minimize.)    
(손실 함수: categorical_crossentropy(손실 함수는 가중치 텐서를 학습하는 피드백 신호로 사용되며, 훈련 단계에서는 이 값을 최소화하려고 함)
- This reduction of the loss happens via mini-batch stochastic gradient descent.   
(이러한 loss의 감소는 mini-batch SGD를 통해 발생함)
- The exact rules governing a specific use of gradient descent are defined by the rmsprop optimizer passed as the first argument.  
(경사하강법의 특정 사용을 제어하는 정확한 규칙은 첫 번째 인수로 전달된 rmsprop optimizer에 의해 정의됨)

- 다중 분류) 손실함수: 'categorical_crossentropy' | 출력층: softmax + one-hot labels
- 이진 분류) 손실함수: 'binary_crossentropy' | 출력층: sigmoid + binary labels
- 회귀) 손실함수: 'mean_squared_error', 'mae' | 출력층: 출력층에 활성화 함수 없음

## Training :

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 7s 13ms/step - loss: 0.2382 - accuracy: 0.9272
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0892 - accuracy: 0.9728
Epoch 3/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0576 - accuracy: 0.9820
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0400 - accuracy: 0.9876
Epoch 5/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0294 - accuracy: 0.9905


- What happens when you call fit: the network will start to iterate on the training data in mini-batches of 128 samples, 5 times over (each iteration over all the training data is called an epoch).   
(fit을 하면 어떤 일이 발생하나: 네트워크는 128개 샘플의 미니 배치로 훈련 데이터를 5회 이상 반복하기 시작함(모든 훈련 데이터에 대한 각 반복을 에포크라고 함))
- At each iteration, the network will compute the gradients of the weights with regard to the loss on the batch, and update the weights accordingly.   
(각 iteration에서 네트워크는 배치 손실에 대한 가중치 기울기를 계산하고 그에 따라 가중치를 업데이트 함)
- After these 5 epochs, the network will have performed 2,345 gradient updates (469 per epoch), and the loss of the network will be sufficiently low that the network will be capable of classifying handwritten digits with high accuracy.  
<font color="orange">**(5epoch이 진행되면, 네트워크는 2,345회의 gradient 업데이트(에포크당 469개)를 수행하게 되며 네트워크 손실은 네트워크가 손으로 쓴 숫자를 높은 정확도로 분류할 수 있을 만큼 충분히 낮아질 것임)**</font>



## Test (Inference):

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0776 - accuracy: 0.9774
test_acc: 0.977400004863739


## Summary

- <font color="orange">**학습(Learning, Training)은 훈련 데이터 샘플과 그에 상응하는 타깃이 주어졌을 때  
손실 함수를 최소화하는 모델 파라미터 조합을 찾는 것을 의미함**</font>


- <font color="orange">**Mini-batch SGD :   
데이터 샘플과 타깃의 배치를 랜덤하게 뽑고 이 배치에서 손실에 대한 파라미터의 그래디언트를 계산함으로써 학습이 진행됨  
신경망의 파라미터는 그래디언트의 반대 방향으로 조금씩(학습률에 의해 정의된 크기만큼) 움직임.**</font>

-  
전체 학습 과정은 신경망이 **미분 가능**한 텐서 연산으로 연결되어 있기 때문에 가능함  
현재 파라미터와 배치 데이터를 그래디언트 값에 매핑해 주는 그래디언트 함수를 구성하기 위해 미분의 연쇄 법칙을 사용함

-
<U>**손실과 옵티마이저**</U> 이 두 가지는 네트워크에 데이터를 주입하기 전에 정의되어야 함.

* **손실**은 훈련하는 동안 최소화해야 할 양이므로 해결하려는 문제의 성공을 측정하는 데 사용함.

* **옵티마저**는 손실에 대한 그래디언트가 파라미터를 업데이트하는 정확한 방식을 정의함.
예를 들어 RMSprop, Adam optimizer 등이 주로 사용됨.[Keras-optimizers](https://keras.io/api/optimizers/).  

참고)   
RMSProp (Root Mean Square Propagation)
- 각 파라미터의 기울기 변화량에 따라 학습률을 자동 조절하는 방식으로, 만일 변화가 큰 파라미터는 학습률을 줄이고, 작은 파라미터는 키워줌
- RNN이나 순차 데이터에 효과적임

Adam (Adaptive Moment Estimation)
- Momemtum + RMSProp의 장점을 결합한 알고리즘
- 학습률 자동으로 조절하면서 관성 효과까지 적용
- 초기 설정에 민감하지 않고, 다양한 문제에 잘 작동하기 때문에 딥러닝에서 가장 널리 쓰임